In [3]:
import pandas as pd
from datetime import timedelta, date
from get_data import daterange, get_data, parse_info

In [40]:
cities = pd.read_csv('data/ongoing_cities.csv', header = False)

In [41]:
cities

,state_code,city_code,city
0,USA-MA,USA-BOS-BOT,Boston
1,USA-IL,USA-CHI-CHH,Chicago
2,USX-TX,USA-DAL-DLL,Dallas
3,USA-MI,USA-DET-DET,Detroit
4,USA-NV,USA-LAS-LAS,Las Vegas
5,USA-CA,USA-LAX-LSN,Los Angeles
6,USA-FL,USA-MIA-MIF,Miami
7,USA-MN,USA-MIN-MIA,Minneapolis
8,USA-NY,USA-NYM-NEY,New York City-Manhattan
9,USA-FL,USA-ORL-ORL,Orlando


In [3]:
url = 'https://s3-us-west-2.amazonaws.com/hoodsjson/USA-CA/USA-SFO-SNF/2014-09-30/0.html'

In [4]:
json_df = pd.read_json(url)    

In [5]:
condition = json_df['source'] == 'CRAIG'
craig_df = json_df[condition]
craig_df = craig_df.reset_index()
del craig_df['index']

In [6]:
cols = ['heading', 'body', 'price', 'lat', 'long', 'region', 'neighborhood', 
        'beds', 'baths', 'parking', 'washer_dryer']

In [7]:
results_df = parse_info(craig_df, headers = cols)

In [9]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 12 columns):
heading         498 non-null object
body            498 non-null object
price           495 non-null float64
lat             498 non-null object
long            498 non-null object
region          497 non-null object
neighborhood    490 non-null object
beds            497 non-null object
baths           465 non-null object
parking         498 non-null object
washer_dryer    498 non-null object
date            498 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 50.6+ KB


In [45]:
start = date(2015, 6, 4)
end = date(2015, 6, 10)

In [46]:
start

datetime.date(2015, 6, 4)

In [47]:
results_df

,heading,body,price,lat,long,region,neighborhood,beds,baths,parking,washer_dryer,date
0,Affordable Newly Remodeled Great Loc. 3 Bed...,"\n Hi there,\nI have a fully detached bright &...",2800,37.73154,-122.4971,sfc,sunset / parkside,3,2,0,1,2014-09-29 07:57:15
1,2 BR Condo at Telegraph Landing with Stunning ...,"\n Telegraph Landing, 2-level condo located on...",5200,37.80392,-122.4043,sfc,north beach / telegraph hill,2,2,4,1,2014-09-29 08:40:52
2,"Cottage Style, Executive Home","\n Flickinger North Valley, Bentley Park Commu...",2750,37.77493,-122.4194,sby,None,3,2,4,1,2014-09-29 08:25:28
3,"2ND FL.,FACE AWAY ST.,REMODELED,LIGHTED, QUIET...","\n 2/1, REMODELED,LIGHTED, QUIET,2 BLOCKS TO U...",3750,37.79023,-122.4083,sfc,financial district,2,1,1,1,2014-09-29 08:57:29
4,"2nd FL., FACES ST.,NEW HWF, BRIGHT, SUNNY, 2 B...","\n 2nd FL., STUDIO FACES ST.,BRIGHT, SUNNY, QU...",2025,37.78995,-122.4104,sfc,nob hill,0,1,1,1,2014-09-29 08:57:36
5,TopFlatwBreathtakenViews inSafeNeighborhd nrBA...,\n $3100 2BR 1BA upper Duplex w. incredible vi...,3100,37.73158,-122.4397,sfc,glen park,2,1,1,1,2014-09-29 08:57:24
6,"2ND FL.,FACE AWAY ST.,REMODELED,LIGHTED, QUIET...","\n 2/1, REMODELED,LIGHTED, QUIET,2 BLOCKS TO U...",3750,37.79023,-122.4083,sfc,financial district,2,1,1,1,2014-09-29 08:57:30
7,"BASEMENT LVL, 2/1, REMDLD APT.,QUIET, LIGHTED,...","\n BASEMENT LVL, 2/1, REMDLD APT.,QUIET, LIGHT...",3695,37.79023,-122.4083,sfc,nob hill,2,1,1,1,2014-09-29 08:57:30
8,"Contemporary Pacific Heights 2 bedroom, 2 bath...",\n This is a Contemporary Pacific Heights 2 be...,3500,37.79275,-122.4253,sfc,pacific heights,2,2,4,1,2014-09-29 08:13:15
9,Furnished One Bedroom on Mission Street With Wifi,\n This unit brings San Francisco to your door...,1950,37.76505,-122.4197,NaN,Mission Street,1,1,1,1,2014-09-29 09:21:26


In [69]:
sf = 'sf'
results_df.to_csv('%s_sample.csv' %sf, encoding='utf-8', index=False) 

In [71]:
pd.read_csv('sf_sample.csv', header=False)

,heading,body,price,lat,long,region,neighborhood,beds,baths,parking,washer_dryer,date
0,Affordable Newly Remodeled Great Loc. 3 Bed...,"\n Hi there,\nI have a fully detached bright &...",2800,37.731542,-122.497125,sfc,sunset / parkside,3,2,0,1,2014-09-29 07:57:15
1,2 BR Condo at Telegraph Landing with Stunning ...,"\n Telegraph Landing, 2-level condo located on...",5200,37.803917,-122.404284,sfc,north beach / telegraph hill,2,2,4,1,2014-09-29 08:40:52
2,"Cottage Style, Executive Home","\n Flickinger North Valley, Bentley Park Commu...",2750,37.774930,-122.419420,sby,NaN,3,2,4,1,2014-09-29 08:25:28
3,"2ND FL.,FACE AWAY ST.,REMODELED,LIGHTED, QUIET...","\n 2/1, REMODELED,LIGHTED, QUIET,2 BLOCKS TO U...",3750,37.790227,-122.408251,sfc,financial district,2,1,1,1,2014-09-29 08:57:29
4,"2nd FL., FACES ST.,NEW HWF, BRIGHT, SUNNY, 2 B...","\n 2nd FL., STUDIO FACES ST.,BRIGHT, SUNNY, QU...",2025,37.789955,-122.410399,sfc,nob hill,0,1,1,1,2014-09-29 08:57:36
5,TopFlatwBreathtakenViews inSafeNeighborhd nrBA...,\n $3100 2BR 1BA upper Duplex w. incredible vi...,3100,37.731584,-122.439743,sfc,glen park,2,1,1,1,2014-09-29 08:57:24
6,"2ND FL.,FACE AWAY ST.,REMODELED,LIGHTED, QUIET...","\n 2/1, REMODELED,LIGHTED, QUIET,2 BLOCKS TO U...",3750,37.790227,-122.408251,sfc,financial district,2,1,1,1,2014-09-29 08:57:30
7,"BASEMENT LVL, 2/1, REMDLD APT.,QUIET, LIGHTED,...","\n BASEMENT LVL, 2/1, REMDLD APT.,QUIET, LIGHT...",3695,37.790227,-122.408251,sfc,nob hill,2,1,1,1,2014-09-29 08:57:30
8,"Contemporary Pacific Heights 2 bedroom, 2 bath...",\n This is a Contemporary Pacific Heights 2 be...,3500,37.792748,-122.425254,sfc,pacific heights,2,2,4,1,2014-09-29 08:13:15
9,Furnished One Bedroom on Mission Street With Wifi,\n This unit brings San Francisco to your door...,1950,37.765051,-122.419669,NaN,Mission Street,1,1,1,1,2014-09-29 09:21:26


In [72]:
!ls

README.md             get_data.py           project_notes         sf_sample.csv
data                  get_data.pyc          rental_data_eda.ipynb


In [73]:
!cd data/csvs

In [74]:
!ls

README.md             get_data.py           project_notes         sf_sample.csv
data                  get_data.pyc          rental_data_eda.ipynb


In [75]:
!cd data

In [76]:
!;s

/bin/sh: -c: line 0: syntax error near unexpected token `;'
/bin/sh: -c: line 0: `;s'


In [77]:
!ls

README.md             get_data.py           project_notes         sf_sample.csv
data                  get_data.pyc          rental_data_eda.ipynb


In [78]:
cd data

/Users/jonoleson/galvanize_work/PriceMyRental/data


In [79]:
!ls

csvs               ongoing_cities.csv


In [80]:
cd csvs

/Users/jonoleson/galvanize_work/PriceMyRental/data/csvs


In [81]:
ls

USA-ATL-ATA_USA-GA.csv  USA-LAX-LSN_USA-CA.csv  USA-SAC-SAC_USA-CA.csv
USA-AUT-AUS_USA-TX.csv  USA-MIA-MIF_USA-FL.csv  USA-SAN-SAD_USA-CA.csv
USA-BOS-BOT_USA-MA.csv  USA-MIN-MIA_USA-MN.csv  USA-SAT-SAT_USA-TX.csv
USA-CHI-CHH_USA-IL.csv  USA-NYM-NEY_USA-NY.csv  USA-SEA-SET_USA-WA.csv
USA-DAL-DLL_USX-TX.csv  USA-ORL-ORL_USA-FL.csv  USA-SFO-OAA_USA-CA.csv
USA-DEN-DEN_USA-CO.csv  USA-PHI-PHI_USA-PA.csv  USA-SFO-SNF_USA-CA.csv
USA-DET-DET_USA-MI.csv  USA-PHX-PHO_USA-AZ.csv  USA-WAS-WSD_USA-DC.csv
USA-HOU-HOU_USA-TX.csv  USA-POR-POR_USA-OR.csv
USA-LAS-LAS_USA-NV.csv  USA-RAL-RAL_USA-NC.csv


In [5]:
atl_df = pd.read_csv('data/csvs/USA-ATL-ATA_USA-GA.csv', header=False)

In [6]:
atl_df

,baths,beds,body,date,heading,lat,long,neighborhood,parking,price,region,washer_dryer
0,1,1,"\n Located in North Atlanta, the Veridian at S...",2015-05-28 08:00:13,"Public Transportation, Microwave, Extra Storage",33.979500,-84.341100,Atlanta,0,850,atl,1
1,1,1,\n Welcome home to The Pointe at Canyon Ridge....,2015-05-28 08:20:55,Clubhouse Easy Access to Freeways Controlled...,33.979632,-84.350661,8350 Roswell Rd,0,734,nat,1
2,1,1,\n These 300 lofts can have a 2 level floor pl...,2015-05-28 09:09:05,Large 1 Bed/1.5 Bath Live in Atlantic Station,33.791863,-84.395707,Midtown Atlanta,0,1910,atl,1
3,2,3,\n master suite sheets first floor Freezer in ...,2015-05-28 09:14:23,3 Bedrm 2.5 Bath 2 Car Garage..single Family,33.781700,-84.387330,otp east,0,865,eat,1
4,1,1,\n Welcome to Embarcadero Club!\n2210 Sullivan...,2015-05-28 10:15:31,Come Get Tanned At Our Sparkling Pool Embarca...,33.688910,-84.303310,college park,0,584,atl,1
5,1,1,"\n Located in North Atlanta, the Veridian at S...",2015-05-28 10:00:10,"Hard Surface Flooring, Lighted Tennis Courts, ...",33.979500,-84.341100,Atlanta,0,1032,atl,1
6,NaN,0,\n Rent-To-Own Condos Starting At $695 Monthly...,2015-05-28 10:21:48,RENT-TO-OWN CONDOS IN ATLANTA GA,33.749000,-84.387980,Atlanta GA,0,695,atl,1
7,2,2,\n This 2 Bedroom/2.5 Bathroom Townhouse locat...,2015-05-28 10:25:56,"2 BD, 2.5 Bath Townhouse for rent",33.900712,-84.261833,doraville,0,900,nat,1
8,1,2,\n Welcome home to The Pointe at Canyon Ridge....,2015-05-28 10:20:56,Cable Ready Laundry Facility Disability Access,33.979632,-84.350661,8350 Roswell Rd,0,978,nat,1
9,2,3,\n upper deck black spotless his and her fabul...,2015-05-28 09:37:40,Large 3 Bed / 2 Bath- Peaceful Home,33.781700,-84.387330,otp south,0,845,wat,1


In [1]:
k = 0
while k < 10:
    print k
    k += 1

0
1
2
3
4
5
6
7
8
9
